In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import keras
print(keras.__version__)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

import utils

## Read in images and labels

In [ ]:
tgt = '/path/to/processed_images'

imgs, labels = utils.load(tgt)

In [ ]:
# Print a few sample images to ensure everything is read correctly

from PIL import Image

for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(imgs[i*113])
    plt.title("labl {}".format(labels[i*113]))

## convert one-hot-encoded value back to string label, and index of predicted class back to string label

In [ ]:
unique_labels, ids = np.unique(labels, return_inverse=True)

def index_to_category(index):
    return unique_labels[index]

def ohe_label_to_category(ohe_label):
    return index_to_category(ohe_label.argmax(0))


## Train test split

In [ ]:
#one hot encoding
#['a', 'b', 'a', 'c', 'b'] 
#=>
#[[1,0,0], [0,1,0], [1,0,0], [0,0,1], [0,1,0]]

mylbs = ['a', 'b', 'a', 'c', 'b', 'd'] 
print(type(mylbs))
myuniques, myids = np.unique(mylbs, return_inverse=True)
print(myuniques)
print(myids)
mylbs_ohe = np_utils.to_categorical(myids, len(myuniques))
print(mylbs_ohe)

In [ ]:
# [1,0,0] => 'a'
myohe = array([1,0,0])
myidx = myohe.argmax(0)
print(myidx)
mylb = myuniques[myidx]
print(mylb)

In [ ]:
#print(type(imgs))
#print(type(imgs[0]))
#print(imgs[0].shape)
print(len(imgs[0].flatten()))
#print(type(array(imgs)))
#print(array(imgs).shape)

In [ ]:
from numpy import array
from sklearn.model_selection import train_test_split

labels_ohe = utils.one_hot_encode_object_array(labels)

x_train, x_test, y_train, y_test = train_test_split(imgs, labels_ohe, random_state = 42)
# tuple to numpy array
img_train, img_test, label_train, label_test = array(x_train), array(x_test), array(y_train), array(y_test)

print("Training matrix shape", img_train.shape)
print("Testing matrix shape", img_test.shape)

In [ ]:
# print a few training image to make sure everything is correct
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(Image.fromarray(img_train[i*87].astype('uint8'),'RGB'))
    plt.title("labl {}".format(ohe_label_to_category(label_train[i*87])))

In [ ]:
# print a few test image to make sure everything is correct
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(Image.fromarray(img_test[i*35].astype('uint8'),'RGB'))
    plt.title("labl {}".format(ohe_label_to_category(label_test[i*35])))

In [ ]:
# normalize 0-255 to 0-1, it might help training
img_train = img_train.astype('float32')
img_train /= 255
img_test = img_test.astype('float32')
img_test /= 255

## Build the CNN

In [ ]:
model = Sequential()
model.reset_states()
model.add(Conv2D(filters=32, kernel_size=5, input_shape=(128,128,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation = 'softmax'))

for i in range(len(model.layers)):
    print(model.layers[i].input)
    print(model.layers[i].output)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [ ]:
model.fit(img_train, label_train, batch_size=12, epochs=10, validation_split=0.1, verbose=1)

## Evaluate CNN

In [ ]:
loss, accuracy = model.evaluate(img_test, label_test, verbose=0)
print(accuracy)

In [ ]:
#this is in 0-9 classes, the index of unique labels
predicted_classes = model.predict_classes(img_test)  

In [ ]:
# convert one-hot-encoded test label to indices
idx_from_ohe = lambda t: t.argmax(0)
idx_label_test = np.array([idx_from_ohe(ti) for ti in label_test])

# Check which items we got right / wrong
correct_indices = np.nonzero(predicted_classes == idx_label_test)[0]
incorrect_indices = np.nonzero(predicted_classes != idx_label_test)[0]

In [ ]:
img_test *= 255

plt.rcParams['figure.figsize'] = (12,12)
plt.figure()
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(Image.fromarray(img_test[correct].astype('uint8'),'RGB'))
    plt.title("L:{}/P:{}".format(index_to_category(idx_label_test[correct]), index_to_category(predicted_classes[correct])))

plt.figure()
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(Image.fromarray(img_test[incorrect].astype('uint8'),'RGB'))
    plt.title("L:{}/P:{}".format(index_to_category(idx_label_test[incorrect]), index_to_category(predicted_classes[incorrect])))

## Save the model

In [ ]:
from keras.models import model_from_json

localModelPath = '/path/to/saved_model'

model_json = model.to_json()
with open(localModelPath + '/kerasModel.json', "w") as json_file:
    json_file.write(model_json)

model.save_weights(localModelPath + '/kerasModel.h5")
